<div style="text-align: center;">
    <img src="https://www.validusrm.com/wp-content/uploads/2022/02/validus-logo.png" alt="Validus Logo" style="width:300px; margin-bottom:30px;">
    <br>
    <h1>Validus Risk Management Case Study</h1>
    <br>
    <h2>Quantitative Research Analyst</h2>
    <br><br><br><br><br><br><br>
    <p><strong>Submitted by:</strong> Armen Bzdikian</p>
    <br>
    <p><strong>Date:</strong> September 27, 2024</p>
</div>


# Table of Contents
- [1. Introduction](#1-introduction)
- [2. Data Loading and Preparation](#2-data-loading-and-preparation)
  - [2.1. Prepare Options Data](#21-prepare-options-data)
  - [2.2. Prepare Spot Prices](#22-prepare-spot-prices)
- [3. Helper Functions](#3-helper-functions)
  - [3.1. Determine the Third Friday of a Month](#31-determine-the-third-friday-of-a-month)
  - [3.2. Get Nth Expiry Date](#32-get-nth-expiry-date)
  - [3.3. Black-Scholes Call Option Price](#33-black-scholes-call-option-price)
  - [3.4. Implied Volatility Calculation](#34-implied-volatility-calculation)
  - [3.5. Initialize Portfolio](#35-initialize-portfolio)
  - [3.6. Settle Expired Options](#36-settle-expired-options)
  - [3.7. Trade New Options](#37-trade-new-options)
  - [3.8. Finite Difference Delta](#38-finite-difference-delta)
  - [3.9. Mark-to-Market Options and Compute Metrics](#39-mark-to-market-options-and-compute-metrics)
  - [3.10. Update Portfolio Metrics](#310-update-portfolio-metrics)
  - [3.11. Advance to Next Trading Day](#311-advance-to-next-trading-day)
- [4. Strategy Implementations](#4-strategy-implementations)
  - [4.1. Strategy 1](#41-strategy-1)
  - [4.2. Strategy 2](#42-strategy-2)
  - [4.3. Strategy 3](#43-strategy-3)
- [5. Comparison of Greek Exposure of Strategy I and Strategy II](#5-comparison-of-greek-exposure-of-strategyI-and-strategyII)
- [6. Conclusion](#6-conclusion)
        

# 1. Introduction

This notebook implements and analyzes three options trading strategies using historical data. We will:

1. *Load and Prepare Data*: Import and clean options and spot price data.
2. *Define Helper Functions*: Utilities for date calculations and delta approximations, Track cash balances, option positions, and portfolio metrics.
3. *Implement Strategies*:
    - *Strategy I*: Sell a 1% OTM one-month call option.
    - *Strategy II*: Sell a 1% OTM two-month call option and compute implied volatility.
    - *Strategy III*: Sell a 1% OTM two-month call option and buy a 2% OTM two-month call option, computing portfolio delta.
4. *Visualize Results*: Plot portfolio performance metrics over time.
5. *StrategyI and StrategyII Difference*: discuss the difference in the order of magnitude between Strategy I and Strategy II Greeks (exposure).
6. *Conclusion*: Summarize findings and insights.

### Importing packages and Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import calendar
from scipy.stats import norm
import numpy as np
from scipy.optimize import brentq

# 2. Data Loading and Preparation

Accurate and well-structured data is the cornerstone of any quantitative trading strategy. In this section, we'll:

1. *Load Options Data*: Import historical options data from an Excel file.
2. *Prepare Options Data*: Clean and format the data for analysis.
3. *Prepare Spot Prices*: Extract and organize the underlying asset's spot prices.

- The options data is stored in an Excel file named 'SPX_Monthly_Option_data_300121_300421__002_.xlsx'.
- The dataset includes necessary columns such as date, expiration, strike, bid, ask, call/put, and adjusted close.

*Note*: Ensure that the Excel file is placed in the same directory as this notebook or provide the correct file path.

In [2]:

file_path = 'SPX_Monthly_Option_data_300121_300421__002_.xlsx'
options_df = pd.read_excel(file_path)


## 2.1. Prepare Options Data

Data rarely comes in a perfect format. Cleaning and formatting are essential to ensure that subsequent analyses and computations are accurate. In this step, we'll:

1. *Convert Date Columns*: Ensure that date and expiration are in datetime format.
2. *Convert Numerical Columns*: Ensure that strike, bid, ask, and adjusted close are of float type.
3. *Clean Categorical Columns*: Standardize the call/put column for consistency.
4. *Handle Missing Data*: Address any missing or inconsistent entries to maintain data integrity.


In [3]:
def prepare_options_data(options_df):
    
    # Convert date columns to datetime
    options_df['date'] = pd.to_datetime(options_df['date'])
    options_df['expiration'] = pd.to_datetime(options_df['expiration'])
    
    # Convert numerical columns
    numerical_columns = ['strike', 'ask', 'bid', 'adjusted close']
    for col in numerical_columns:
        options_df[col] = options_df[col].astype(float)
    
    # Clean the 'call/put' column
    options_df['call/put'] = options_df['call/put'].str.strip().str.upper()
    
    return options_df

# Prepare the data
options_df = prepare_options_data(options_df)


## 2.2. Prepare Spot Prices

The spot price represents the current price of the underlying asset (e.g., S&P 500 index). Accurate spot prices are crucial for:

- *Calculating Option Greeks*: Deltas, implied volatilities, etc., rely on the spot price.
- *Determining Moneyness*: Whether an option is In-The-Money (ITM), At-The-Money (ATM), or Out-Of-The-Money (OTM) depends on the spot price relative to the strike price.

*Steps*:

1. *Extract Unique Spot Prices*: From the adjusted close column, ensure that each trading day has a corresponding spot price.
2. *Handle Missing Days*: Create a complete sequence of trading days and forward-fill any missing spot prices to maintain continuity.
3. *Validate Spot Price Data*: Ensure that spot prices are accurate and free from anomalies.


In [ ]:
def prepare_spot_prices(options_df):
    
    # Extract unique dates and corresponding spot prices
    spot_df = options_df[['date', 'adjusted close']].drop_duplicates()
    spot_df.rename(columns={'adjusted close': 'spot_price'}, inplace=True)
    
    # Create a complete range of business days
    all_dates = pd.date_range(start=spot_df['date'].min(), end=spot_df['date'].max(), freq='B')
    
    # Reindex to include all trading days and forward-fill missing spot prices
    spot_df = spot_df.set_index('date').reindex(all_dates).fillna(method='ffill').reset_index()
    spot_df.rename(columns={'index': 'date'}, inplace=True)
    
    return spot_df

# Prepare spot prices
spot_df = prepare_spot_prices(options_df)
spot_df.head()

# 3. Helper Functions

Helper functions perform specific tasks that support the main strategy implementation. These utilities handle date calculations, option expirations, delta approximations, and other essential operations.

### Key Helper Functions:

1. *third_friday*: Determines the third Friday of a given month and year, commonly used as the standard expiration date for options.
2. *get_nth_expiry*: Finds the expiration date n months ahead based on the third Friday convention.
3. *black_scholes_call*: Computes the price of a European call option using the Black-Scholes formula.
4. *implied_volatility_call*: Estimates the implied volatility of a European call option by inverting the Black-Scholes formula using numerical methods
5. *Initialize Portfolio*: Set up a DataFrame to monitor cash balances, option positions, and overall portfolio value.
6. *Settle Expired Options*: Handle the settlement of options that reach expiration.
7. *Trade New Options*: Execute trades based on defined strategies, update cash balances, and track active positions.
8. *Mark-to-Market Valuation*: Assess the current value of active options and compute relevant metrics like delta 
9.  *Finite Difference Method*:  The *finite difference method* approximates delta by observing the change in option prices over consecutive trading days.
10. *Update Portfolio Metrics*: Log daily portfolio performance metrics for analysis.
11. *Advance Trading Days*: Move the analysis forward to the next trading day, accounting for weekends and holidays.

## 3.1. Determine the Third Friday of a Month

Options typically expire on the *third Friday* of their expiration month. Identifying this date is crucial for managing option expirations accurately.

In [5]:
def third_friday(year, month):
    c = calendar.Calendar(firstweekday=calendar.SUNDAY)
    monthcal = c.monthdatescalendar(year, month)
    third_friday_date = [day for week in monthcal for day in week if \
                         day.weekday() == calendar.FRIDAY and \
                         day.month == month][2]
    return pd.Timestamp(third_friday_date)

## 3.2. Get Nth Expiry Date

Often, strategies require options with expiration dates several months ahead. The get_nth_expiry function determines the expiration date that is n months ahead of the current date, adhering to the third Friday rule.

*Logic*:

1. *Iterate Through Months*: Increment the month while handling year transitions.
2. *Identify Expiration Dates*: Use third_friday to find the expiration date for each month.
3. *Select the nth Expiry*: Collect and return the `n`th expiration date that is after the current date.

In [6]:
def get_nth_expiry(current_date, n):
    expiry_dates = []
    year = current_date.year
    month = current_date.month
    while len(expiry_dates) < n:
        expiry = third_friday(year, month)
        if expiry > current_date:
            expiry_dates.append(expiry)
        month += 1
        if month > 12:
            month = 1
            year += 1
    return expiry_dates[-1]

## 3.3. Black-Scholes Call Option Price

The **Black-Scholes model** is a widely used method for pricing European options. It assumes that markets are efficient, there's no arbitrage, and volatility is constant.

**Black-Scholes Formula for Call Options**:

$
C = S \cdot N(d_1) - K \cdot e^{-rT} \cdot N(d_2)
$

Where:

$
d_1 = \frac{\ln(S/K) + (r + 0.5 \sigma^2)T}{\sigma \sqrt{T}}
$
$
d_2 = d_1 - \sigma \sqrt{T}
$

**Parameters**:

- $ S $: Current spot price of the underlying asset.
- $ K $: Strike price of the option.
- $ T $: Time to expiration in years.
- $ r $: Risk-free interest rate (annualized).
- $ \sigma $: Volatility of the underlying asset (annualized).
- $ N(\cdot) $: Cumulative distribution function of the standard normal distribution.




In [7]:
def black_scholes_call(S, K, T, r, sigma):
    
    if T <= 0:
        return max(S - K, 0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T) + 1e-8)
    d2 = d1 - sigma * np.sqrt(T)
    call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return call_price


## 3.4. Implied Volatility Calculation

**Implied Volatility (IV)** represents the market's forecast of a likely movement in an asset's price. Unlike historical volatility, IV is forward-looking and is derived from the market price of an option.

**Objective**:

Given an option's market price, determine the volatility ($ \sigma $) that equates the Black-Scholes price to the observed price.

**Methodology**:

- **Numerical Inversion**: Since the Black-Scholes formula doesn't have a closed-form solution for $ \sigma $, we use numerical methods like Brent's method to find the root of the equation $ C_{\text{BS}}(\sigma) - C_{\text{market}} = 0 $.
  
- **Brent's Method**: A robust root-finding algorithm that combines bisection, secant, and inverse quadratic interpolation methods.

**Considerations**:

- **Market Conditions**: Extreme market conditions may lead to unrealistic IV estimates.
- **Option Liquidity**: Thinly traded options may have unreliable prices, affecting IV accuracy.


In [8]:

def implied_volatility_call(C_market, S, K, T, r, tol=1e-6, max_iterations=100):
    
    if T <= 0:
        return np.nan  # Undefined implied volatility
    
    # Define the objective function for which we seek a root
    def objective_function(sigma):
        return black_scholes_call(S, K, T, r, sigma) - C_market
    
    # Set initial bounds for volatility
    sigma_lower = 1e-6
    sigma_upper = 5.0  # 500% volatility upper bound
    
    try:
        implied_vol = brentq(
            objective_function, sigma_lower, sigma_upper, xtol=tol, maxiter=max_iterations
        )
    except (ValueError, RuntimeError):
        implied_vol = np.nan  # Assign NaN if root-finding fails
    
    return implied_vol


## 3.5. Initialize Portfolio

We need to establish a structure to track our portfolio's state over the strategy period. This includes:

- **Date Tracking**: Each trading day within the strategy timeframe.
- **Cash Balance**: Funds available for trading.
- **Change in Option Value**: Current market value of all active option positions.
- **Portfolio Value**: Sum of cash balance and option value.

**Benefits**:

- **Historical Tracking**: Analyze how the portfolio evolves over time.
- **Performance Assessment**: Evaluate returns, drawdowns, and other performance metrics.
- **Risk Management**: Monitor portfolio sensitivities and exposures.


In [9]:
def initialize_portfolio(start_date, end_date):
    trading_days = pd.date_range(start_date, end_date, freq='B')
    portfolio_df = pd.DataFrame({'date': trading_days})
    portfolio_df['cash_balance'] = 0.0
    portfolio_df['change_in_value'] = 0.0
    portfolio_df['portfolio_value'] = 0.0
    cash_balance = 0.0
    active_options = []
    return portfolio_df, cash_balance, active_options


## 3.6. Settle Expired Options


Options have a finite lifespan and expire on specific dates (in this case the third Friday of the month). Upon expiration, options are settled based on their intrinsic value:

- **Call Options**: Intrinsic value = $ \max(S_T - K, 0) $
- **Put Options**: Intrinsic value = $ \max(K - S_T, 0) $

#### Settlement Scenarios:

- **Short Position**: If holding a short position, the portfolio must pay the intrinsic value.
- **Long Position**: If holding a long position, the portfolio receives the intrinsic value.

#### Process:

1. **Identify Expired Options**: Check if any active options expire on the current date.
2. **Calculate Intrinsic Value**: Based on the spot price at expiration.
3. **Adjust Cash Balance**:
    - **Short Call**: Deduct $ \max(S_T - K, 0) $ from cash.
    - **Long Call**: Add $ \max(S_T - K, 0) $ to cash.
4. **Remove Expired Options**: Clean up active positions.

#### Mathematical Foundation:

$
\text{Intrinsic Value} = \max(S_T - K, 0)
$


In [10]:
def settle_expired_options(current_date, active_options, spot_df):
    cash_flow = 0.0
    expired_today = False

    for opt in active_options.copy():  # Copy to avoid modifying the list while iterating
        if current_date == opt['expiry']:
            # Option is expiring today
            expired_today = True
            
            # Get the spot price at expiry
            S_expiry = spot_df.loc[spot_df['date'] == opt['expiry'], 'spot_price'].values[0]
            intrinsic_value = max(S_expiry - opt['strike'], 0)  # Intrinsic value for a call option
            
            # If short position, pay the intrinsic value (if any)
            if opt['position'] == 'short':
                cash_flow -= intrinsic_value  # Pay intrinsic value if option expires ITM
            elif opt['position'] == 'long':
                cash_flow += intrinsic_value  # Receive intrinsic value if long position
            
            # Remove the expired option from active options
            active_options.remove(opt)

    return cash_flow, expired_today


## 3.7. Trade New Options


Executing trades based on defined strategies involves:

1. **Identifying Target Options**: Selecting options based on moneyness, expiration, and other criteria.
2. **Calculating Premiums**: Determining the cost or proceeds from buying or selling options.
3. **Updating Cash Balances**: Reflecting the financial impact of trades.
4. **Tracking Active Positions**: Adding newly traded options to the list of active positions.

#### Steps:

1. **Determine Target Strike Price**:
    - Based on the current spot price and desired Out-Of-The-Money (OTM) percentage.
    - $ K_{\text{target}} = S_{\text{current}} \times (1 + \text{percent\_otm}) $

2. **Find Closest Strike**:
    - Select the strike price in the dataset closest to $ K_{\text{target}} $.

3. **Execute Trade**:
    - **Short Position**: Receive premium ($ +P_0 $).
    - **Long Position**: Pay premium ($ -P_0 $).

4. **Log the Position**:
    - Store details such as strike price, expiration date, premium, position type, etc.


In [11]:
def trade_new_options(current_date, option_legs, options_df, spot_df, cash_balance):
    
    new_active_options = []
    
    for leg in option_legs:
        position = leg['position']
        months_ahead = leg['months_ahead']
        percent_otm = leg['percent_otm']

        # Get current spot price
        if current_date not in spot_df['date'].values:
            print(f"No spot price available for {current_date}, skipping.")
            continue

        S_current = spot_df.loc[spot_df['date'] == current_date, 'spot_price'].values[0]
        K_target = S_current * (1 + percent_otm)
        K_target = round(K_target)  # Closest rounded strike price

        # Calculate the target expiration date
        expiry_date = get_nth_expiry(current_date, n=months_ahead)

        # Filter options to find the closest strike
        option_data = options_df[
            (options_df['date'] == current_date) &
            (options_df['expiration'] == expiry_date) &
            (options_df['call/put'] == 'C')
        ].copy()

        if option_data.empty:
            print(f"No options available for {current_date} at expiry {expiry_date}, skipping.")
            continue

        # Use .loc[] to avoid SettingWithCopyWarning
        option_data.loc[:, 'strike_diff'] = abs(option_data['strike'] - K_target)
        option_data = option_data.sort_values('strike_diff')
        option_to_trade = option_data.iloc[0]

        K = option_to_trade['strike']
        P0 = (option_to_trade['bid'] + option_to_trade['ask']) / 2  # Mid-point of bid/ask

        # Update cash balance
        if position == 'short':
            cash_balance += P0  # Receive premium for short position
        elif position == 'long':
            cash_balance -= P0  # Pay premium for long position

        # Add the new option to active options
        new_active_options.append({
            'strike': K,
            'expiry': expiry_date,
            'premium': P0,
            'init_date': current_date,
            'last_price': P0,
            'position': position
        })

    return cash_balance, new_active_options


## 3.8. Finite Difference Delta

Delta measures the sensitivity of an option's price to changes in the underlying asset's price. The *finite difference method* approximates delta by observing the change in option prices over consecutive trading days.

*Formula*:

$$
\Delta \approx \frac{C_{\text{today}} - C_{\text{yesterday}}}{S_{\text{today}} - S_{\text{yesterday}}}
$$

*Considerations*:

- *Data Requirements*: Requires accurate tracking of both current and previous option prices and spot prices.
- *Assumptions*: Assumes small changes in the underlying asset's price for accurate approximation.
- *Limitations*: Can be noisy due to market microstructure effects or significant price movements.

*Use Case*:

- When analytical delta (e.g., Black-Scholes delta) is unavailable or for non-standard option types where analytical formulas don't apply.

In [12]:
def finite_difference_delta(option_price_today, option_price_yesterday, S_today, S_yesterday):
    if S_today == S_yesterday:
        return np.nan  # Avoid division by zero
    return (option_price_today - option_price_yesterday) / (S_today - S_yesterday)


## 3.9. Mark-to-Market Options and Compute Metrics

**Mark-to-Market (MTM)** valuation assesses the current market value of all active option positions. Additionally, computing Greeks like delta and metrics like implied volatility provides deeper insights into the portfolio's sensitivity and risk profile.

### Operational Flow of `mark_to_market_options`

1. **Initialization:**
   - Set initial values for total option values, portfolio delta, and implied volatilities.
   $
   \text{option\_values} = 0.0, \quad \text{portfolio\_delta} = 0.0, \quad \text{implied\_vols} = []
   $

2. **Retrieve Spot Price:**
   - Extract spot price $ S $ for the current date from `spot_df`.

3. **Iterate Through Active Options:**
   - For each option in `active_options`:
   
   a. **Identify Option:**
      - Use the strike price and expiration date to filter relevant options data from `options_df`.

   b. **Retrieve Option Price:**
      - Calculate current option price as the midpoint of bid and ask prices:
      $
      P_{\text{current}} = \frac{\text{Bid} + \text{Ask}}{2}
      $
      - Update the option's last known price.

   c. **Update Option Value Change Based on Position:**
      - **Short Position**:
      $
      \text{option\_values} += -P_{\text{current}}
      $
      - **Long Position**:
      $
      \text{option\_values} += P_{\text{current}}
      $

4. **Compute Delta (If Enabled):**
   - **Time to Expiration**:
   $
   T = \frac{\text{Days to Expiration}}{365}
   $
   - **Calculate $ d_1 $** using Black-Scholes:
   $
   d_1 = \frac{\ln\left(\frac{S}{K}\right) + \left(r + 0.5 \sigma^2\right) T}{\sigma \sqrt{T} + \epsilon}
   $
   - Calculate delta:
   $
   \Delta = N(d_1)
   $
   - Adjust portfolio delta based on option position.

5. **Compute Implied Volatility (If Enabled):**
   - Use numerical methods to solve for $ \sigma_{\text{IV}} $ in the Black-Scholes equation.

6. **Return Computed Metrics:**
   - Return option value change, portfolio value, portfolio delta, and implied volatilities


In [13]:
def mark_to_market_options(current_date, active_options, options_df, spot_df, 
                           previous_option_prices, previous_spot_price, risk_free_rate=0, volatility=0.12,
                           compute_delta=False, compute_iv=False, delta_method='black_scholes'):

    option_value_change = 0.0  # Track the change in option values (MTM)
    portfolio_delta = 0.0  # Initialize delta if needed
    implied_vols = []  # List to store implied volatilities for each option
    current_option_price = 0.0  # Default option price if no data is found
    
    # Get the current spot price
    S_series = spot_df.loc[spot_df['date'] == current_date, 'spot_price']
    if S_series.empty:
        return None, None, None, None  # Return None to indicate no option data available
    S = S_series.values[0]
    
    for opt in active_options:
        # Identify the option uniquely by both strike and expiry
        key = (opt['strike'], opt['expiry'])
        
        # Retrieve option data for current date (same expiry, same strike)
        option_data = options_df.loc[
            (options_df['call/put'] == 'C') &
            (options_df['date'] == current_date) &
            (options_df['expiration'] == opt['expiry']) &  # Ensure same expiry
            (options_df['strike'] == opt['strike'])         # Ensure same strike
        ].copy()

        # Get the current option price (mid-point of bid/ask)
        if option_data.empty:
            implied_vols.append(np.nan)  # Skip IV calculation if no option data is found
            continue  # Skip this option if no data is available
        else:
            option_data = option_data.iloc[0]
            current_option_price = (option_data['bid'] + option_data['ask']) / 2



        # Get the previous option price (or the premium if it's the first day)
        previous_option_price = previous_option_prices.get(key, opt['premium'])

        # Calculate the change in the option price
        price_change = current_option_price - previous_option_price

        # Store the current option price for future MTM calculations
        previous_option_prices[key] = current_option_price

        # Adjust the portfolio value based on the position (short or long)
        if opt['position'] == 'short':
            option_value_change -= price_change  # Short position, so price increase reduces value
        elif opt['position'] == 'long':
            option_value_change += price_change  # Long position, price increase adds to value

        # Compute delta if required
        if compute_delta and S is not None:
            K = opt['strike']
            T_days = (opt['expiry'] - current_date).days
            T = T_days / 365.0  # Convert days to years

            if delta_method == 'black_scholes' and T > 0:
                # Black-Scholes delta calculation
                try:
                    d1 = (np.log(S / K) + (risk_free_rate + 0.5 * volatility ** 2) * T) / (volatility * np.sqrt(T))
                    delta = norm.cdf(d1)
                    
                    # Adjust delta based on the position
                    if opt['position'] == 'short':
                        portfolio_delta += -delta
                    elif opt['position'] == 'long':
                        portfolio_delta += delta
                except Exception as e:
                    print(f"Error calculating delta for option {key} on {current_date}: {e}")
                    portfolio_delta = np.nan
            elif delta_method == 'finite_difference' and previous_spot_price is not None:
                # Finite difference delta calculation
                delta = finite_difference_delta(current_option_price, previous_option_price, S, previous_spot_price)
                # Adjust delta based on the position
                if opt['position'] == 'short':
                    portfolio_delta += -delta
                elif opt['position'] == 'long':
                    portfolio_delta += delta
            else:
                portfolio_delta = np.nan  # Set delta to NaN if time to expiry is not valid

        # Compute implied volatility if required
        if compute_iv and S is not None:
            K = opt['strike']
            T_days = (opt['expiry'] - current_date).days
            T = T_days / 365.0  # Convert days to years
            if T > 0:  # Only compute implied volatility if time to expiry is positive
                C_market = current_option_price
                iv = implied_volatility_call(C_market=C_market, S=S, K=K, T=T, r=risk_free_rate)
                
                # Check for invalid implied vol
                if np.isnan(iv):
                    print(f"Failed to compute IV for option {key} on {current_date}.")
                implied_vols.append(iv)
            else:
                implied_vols.append(np.nan)  # Set IV to NaN if time to expiry is not valid
        else:
            implied_vols.append(np.nan)

    # Return option value change, current option price, portfolio delta, and implied volatilities
    return option_value_change, current_option_price, portfolio_delta, implied_vols


## 3.10. Update Portfolio Metrics

After computing the mark-to-market values and any desired metrics (delta, implied volatility), we need to update our portfolio tracking DataFrame accordingly.

**Functionality**:

- **Identify Current Date Entry**: Locate the row in `portfolio_df` corresponding to `current_date`.
- **Update Cash Balance**: Reflect the latest cash balance.
- **Update Change in Option Value**: Reflect the current total value of all active options.
- **Update Mark to Market of Portfolio**: Sum of cash balance and option value.


In [14]:
def update_portfolio(portfolio_df, current_date, cash_balance, option_values):
    
    if current_date == portfolio_df['date'].min():
        # For the first date, the portfolio value is the cash balance + initial option value
        portfolio_value = cash_balance + option_values
    else:
        # For all subsequent days, only the option value change affects the portfolio (cash remains the same)
        previous_dates = portfolio_df.loc[portfolio_df['date'] < current_date]
        
        if previous_dates.empty:
            raise ValueError("No previous trading day found.")
        
        previous_portfolio_value = previous_dates.iloc[-1]['portfolio_value']
        portfolio_value = previous_portfolio_value + option_values

    # Update the portfolio for the current day
    portfolio_df.loc[portfolio_df['date'] == current_date, 'cash_balance'] = cash_balance  # Only changes on option trade days
    portfolio_df.loc[portfolio_df['date'] == current_date, 'change_in_value'] = option_values
    portfolio_df.loc[portfolio_df['date'] == current_date, 'portfolio_value'] = portfolio_value

    return portfolio_df


## 3.11. Advance to Next Trading Day

After processing the current trading day, we need to move forward to the next trading day, ensuring that weekends and holidays are skipped.

**Functionality**:

- **Increment Date**: Move to the next calendar day.
- **Check for Business Day**: Continue incrementing until a business day is found.
- **Return Next Trading Day**: The date to be processed next in the loop.


In [15]:
def advance_to_next_trading_day(current_date):
    """Advances to the next trading day."""
    current_date += pd.DateOffset(days=1)
    while current_date.weekday() >= 5:  # Skip weekends
        current_date += pd.DateOffset(days=1)
    return current_date.normalize()


# 4. Strategy Implementations

With all foundational components in place, we'll now implement the three defined trading strategies. Each strategy involves specific option positions and computations of portfolio metrics.

### Overview of Strategies:

1. **Strategy I**: Sell a 1% Out-Of-The-Money (OTM) one-month call option.
2. **Strategy II**: Sell a 1% OTM two-month call option and compute implied volatility daily.
3. **Strategy III**: Sell a 1% OTM two-month call option and buy a 2% OTM two-month call option, computing portfolio delta daily using the finite difference method.

**Implementation Steps for Each Strategy**:


The `implement_strategy` function serves as the core engine to simulate and manage the portfolio of options. It integrates various helper functions and follows a structured approach to execute different strategies efficiently.

#### 1. Define Option Legs
Each strategy begins by specifying the option legs that dictate the position type (long or short), the number of months ahead for expiration, and the percentage out-of-the-money (OTM) for the options. This forms the core structure of the strategy to determine what options to trade.

#### 2. Execute Strategy
For each trading day within the specified date range, the strategy performs the following steps:

1. **Settle Expired Options**:  
   If any options are expiring on the current day, they are settled. Cash flows are adjusted accordingly depending on the value of the expired option (either a gain or a loss). This is handled using the `settle_expired_options` function.
   
2. **Trade New Options**:  
   On the start date and after an option expires, the strategy trades new options based on the predefined strategy legs. These new options are then added to the list of active positions. This step is managed through the `trade_new_options` function.
   
3. **Mark-to-Market Valuation**:  
   The value of active options is marked to market each day. The function `mark_to_market_options` computes the change in the option’s value, as well as important metrics like the option delta (if enabled) and implied volatility (if enabled). The current value of options is updated daily, allowing for accurate portfolio tracking.

4. **Update Portfolio**:  
   The function `update_portfolio` is used to update the portfolio’s cash balance, option values, and overall portfolio value after each trading day. Additionally, if delta and implied volatility are computed, those values are stored in the portfolio DataFrame as well.

5. **Advance to the Next Trading Day**:  
   The strategy moves forward by identifying the next trading day using `advance_to_next_trading_day`, skipping weekends and holidays to ensure only active market days are processed.

#### 3. Analyze Results
Once the strategy has processed all trading days within the date range, the portfolio DataFrame contains all necessary information for analysis:

- **Portfolio Value**: The total value of the portfolio including cash balance and the current value of open options.
- **Option Delta**: Sensitivity of the portfolio to movements in the underlying asset (if enabled).
- **Implied Volatility**: Market expectations of future volatility for the options (if enabled).


In [16]:
def implement_strategy(options_df, spot_df, start_date, end_date, strategy_legs,
                       risk_free_rate=0, volatility=0.12,
                       compute_delta=False, compute_iv=False, delta_method='black_scholes'):
    
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    # Initialize the portfolio, cash balance, and active options
    portfolio_df, cash_balance, active_options = initialize_portfolio(start_date, end_date)
    previous_option_prices = {}  # Dictionary to track previous option prices
    previous_spot_price = None  # Initialize the previous spot price to None
    
    current_date = start_date
    
    while current_date <= end_date:
        # Settle expired options
        cash_flow, expired_today = settle_expired_options(current_date, active_options, spot_df)
        cash_balance += cash_flow
        
        # Trade new options if needed (on expiry or the start date)
        if expired_today or current_date == start_date:
            cash_balance, new_options = trade_new_options(
                current_date, strategy_legs, options_df, spot_df, cash_balance
            )
            active_options.extend(new_options)
        
        # Mark-to-market active options and compute metrics as required
        option_values, current_option_price, portfolio_delta, implied_vols = mark_to_market_options(
            current_date, active_options, options_df, spot_df,
            previous_option_prices, previous_spot_price,
            risk_free_rate=risk_free_rate, volatility=volatility,
            compute_delta=compute_delta, compute_iv=compute_iv, delta_method=delta_method
        )

        # Store the current spot price to be used in the next day for finite difference delta
        previous_spot_price = spot_df.loc[spot_df['date'] == current_date, 'spot_price'].values[0]

        # Update the portfolio
        portfolio_df = update_portfolio(portfolio_df, current_date, cash_balance, option_values)
        
        # Update implied volatility and delta if required
        if compute_iv:
            portfolio_df.loc[portfolio_df['date'] == current_date, 'implied_volatility'] = implied_vols[0] if implied_vols else np.nan
        
        if compute_delta:
            portfolio_df.loc[portfolio_df['date'] == current_date, 'portfolio_delta'] = portfolio_delta
        
        # Advance to the next trading day
        current_date = advance_to_next_trading_day(current_date)
    
    return portfolio_df


## 4.1. Strategy 1 

In [ ]:
# Define your strategy legs
strategy1_option_legs = [
    {'position': 'short', 'percent_otm': 0.01, 'months_ahead': 1},  # Sell 1% OTM, 1-month call
]

# Define the start and end dates
start_date = '2021-02-01'
end_date = '2021-04-30'

# Implement Strategy I
portfolio_strategy1 = implement_strategy(
    options_df=options_df,
    spot_df=spot_df,
    start_date=start_date,
    end_date=end_date,
    strategy_legs=strategy1_option_legs,
    compute_delta=False,
    compute_iv=False,
    delta_method='finite_difference'
)

# View the result
print(portfolio_strategy1)


plt.figure(figsize=(12, 6))
plt.plot(portfolio_strategy1['date'], portfolio_strategy1['portfolio_value'], label='Portfolio Value')
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.title('Mark to Market Over Time - Strategy I')
plt.legend()
plt.grid(True)
plt.show()


## 4.2. Strategy 2 

In [ ]:
# Strategy 2: Sell 1 unit of 1% OTM two-month call option
strategy2_option_legs = [
    {
        'position': 'short',  # Sell
        'months_ahead': 2,    # Two-month call option
        'percent_otm': 0.01   # 1% OTM
    }
]

# Run Strategy 2
portfolio_strategy2 = implement_strategy(
    options_df=options_df,
    spot_df=spot_df,
    start_date='2021-02-01',
    end_date='2021-04-30',
    strategy_legs=strategy2_option_legs,
    compute_delta=False,  # We don't need delta for strategy 2
    compute_iv=True,   # We need implied volatility for strategy 2
    delta_method='black_scholes'       
)

# View the result for Strategy 2
print(portfolio_strategy2)

plt.figure(figsize=(12, 6))
plt.plot(portfolio_strategy2['date'], portfolio_strategy2['portfolio_value'], label='Portfolio Value')
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.title('Portfolio Over Time - Strategy II')
plt.legend()
plt.grid(True)
plt.show()


In [19]:
def plot_implied_volatility(portfolio_df):
    plt.figure(figsize=(10, 6))
    plt.plot(portfolio_df['date'], portfolio_df['implied_volatility'], label='Implied Volatility', color='orange')
    plt.xlabel('Date')
    plt.ylabel('Implied Volatility')
    plt.title('Implied Volatility Over Time')
    plt.grid(True)
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(spot_df['date'], spot_df['spot_price'], label='Spot Price', color='blue')

# Labeling the graph
plt.title("S&P 500 Spot Prices Over Time")
plt.xlabel("Date")
plt.ylabel("Spot Price")
plt.grid(True)
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
clean_portfolio_strategy2= portfolio_strategy2.dropna(subset=['implied_volatility'])
plot_implied_volatility(clean_portfolio_strategy2)

## 4.3. Strategy 3 

In [ ]:
# Strategy 3: Sell 1 unit of 1% OTM call and buy 1 unit of 2% OTM call
strategy3_option_legs = [
    {
        'position': 'short',  # Sell
        'months_ahead': 2,    # Two-month call option
        'percent_otm': 0.01   # 1% OTM
    },
    {
        'position': 'long',   # Buy
        'months_ahead': 2,    # Two-month call option
        'percent_otm': 0.02   # 2% OTM
    }
]

# Run Strategy 3
portfolio_strategy3 = implement_strategy(
    options_df=options_df,
    spot_df=spot_df,
    start_date='2021-02-01',
    end_date='2021-04-30',
    strategy_legs=strategy3_option_legs,
    compute_delta=True,   # We need delta for strategy 3
    compute_iv=False,
    delta_method='black_scholes'
          # Implied volatility is not required for strategy 3
)

# View the result for Strategy 3
print(portfolio_strategy3)


In [ ]:
def plot_portfolio_delta(portfolio_df):
    plt.figure(figsize=(10, 6))
    plt.plot(portfolio_df['date'], portfolio_df['portfolio_delta'], label='Portfolio Delta', color='blue')
    plt.xlabel('Date')
    plt.ylabel('Delta')
    plt.title('Portfolio Delta Over Time')
    plt.grid(True)
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
clean_portfolio_strategy3 = portfolio_strategy3[portfolio_strategy3['portfolio_delta'] != 0]
plot_portfolio_delta(clean_portfolio_strategy3)


# 5. Comparison of Greek Exposure of StrategyI and StrategyII

### **Strategy I (1-Month 1% OTM Call Option)**
- **Shorter Time to Maturity**: Higher sensitivity to delta near expiration.
- **Lower Vega**: Less sensitive to volatility changes due to short duration.
- **Rapid Delta Decay**: Delta changes faster as the option approaches expiration.
- **Higher Gamma**: More frequent delta adjustments due to underlying price movements.
- **Faster Theta (Time Decay)**: Option loses value more quickly as expiration nears.

### **Strategy II (2-Month 1% OTM Call Option)**
- **Longer Time to Maturity**: Slower delta changes, more stable early in the option’s life.
- **Higher Vega**: More sensitive to volatility changes due to longer duration.
- **Slower Delta Decay**: Delta adjusts more gradually until closer to expiration.
- **Lower Gamma**: Delta is less sensitive to small price movements in the underlying.
- **Slower Theta Decay**: Time decay accelerates only closer to expiration.

1. **Delta**: 
   - Strategy I has more rapid delta changes near expiration.
   - Strategy II has slower delta changes initially.
   
2. **Vega**: 
   - Strategy II has higher vega, making it more sensitive to volatility shifts.
   
3. **Gamma**: 
   - Strategy I has higher gamma, making delta more responsive to underlying price changes.
   
4. **Theta**: 
   - Strategy I loses value faster due to accelerated time decay.



# 6. Conclusion

Throughout this analysis, we've implemented and evaluated three distinct options trading strategies using historical data. Here's a summary of our findings and insights:

### Strategy I: Sell a 1% OTM One-Month Call Option

- **Performance**: Consistently generated premium income, reflected in the steady increase of portfolio value.
- **Risk Profile**: Limited upside due to the OTM strike price; potential losses if the underlying asset's price surged significantly beyond the strike price.
- **Insights**:
    - Selling OTM options can be an effective income-generating strategy with controlled risk.
    - The strategy's success is influenced by the underlying asset's price stability and time decay.

### Strategy II: Sell a 1% OTM Two-Month Call Option and Compute Implied Volatility

- **Performance**: Similar to Strategy I, but with extended duration, leading to higher premium income.
- **Implied Volatility**: Provided valuable insights into market expectations of future volatility. Elevated IV periods corresponded with increased option premiums.
- **Insights**:
    - Longer-duration options capture more premium but are exposed to more market uncertainty.
    - Monitoring implied volatility can inform adjustments to the strategy, such as rolling options or hedging positions.

### Strategy III: Sell a 1% OTM Two-Month Call Option and Buy a 2% OTM Two-Month Call Option, Computing Portfolio Delta

- **Performance**: Implemented a **call spread**, limiting both potential gains and losses.
- **Portfolio Delta**: Maintained near-neutral delta, reducing sensitivity to underlying asset price movements.
- **Insights**:
    - **Risk Mitigation**: The long call at a higher strike acts as a hedge against unlimited losses from the short call.
    - **Profit Potential**: Capped due to the simultaneous long and short positions.
    - **Delta Neutrality**: Facilitates a more controlled exposure to the underlying asset's price changes.

### General Observations

- **Data Integrity**: Accurate and comprehensive data preparation is crucial for reliable strategy implementation and analysis.
- **Metrics Tracking**: Monitoring portfolio metrics like delta and implied volatility enhances risk management and strategic decision-making.
- **Visualization**: Graphical representations aid in understanding strategy performance and market dynamics.

### Future Enhancements

1. **Incorporate Other Greeks**: Extend analysis to include gamma, theta, vega, and rho for a more comprehensive risk profile.
2. **Dynamic Volatility Models**: Instead of assuming constant volatility, use historical data or models like GARCH to estimate time-varying volatility.
3. **Risk Management Techniques**: Introduce stop-loss mechanisms, position sizing rules, hedging the greeks and diversification strategies to further manage risk.
4. **Real-Time Data Integration**: Connect to live data feeds for real-time strategy execution and monitoring.

By systematically building and analyzing these strategies, we've established a solid foundation for further exploration and refinement in options trading. Continuous learning and adaptation are key to navigating the complexities of financial markets effectively.


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(portfolio_strategy1['date'], portfolio_strategy1['portfolio_value'], label="Strategy 1", color='blue')
plt.plot(portfolio_strategy2['date'], portfolio_strategy2['portfolio_value'], label="Strategy 2", color='orange')
plt.plot(portfolio_strategy3['date'], portfolio_strategy3['portfolio_value'], label="Strategy 3", color='green')
plt.title('Portfolio Value Comparison for All Strategies')
plt.xlabel('Date')
plt.ylabel('Portfolio Value')
plt.legend()
plt.grid(True)
plt.show()
